<a href="https://colab.research.google.com/github/efe-akpinar/sleep-quality-analysis/blob/main/Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import xml.etree.ElementTree as ET

# Orginizing the Data Collected From Health **App**

In [ ]:


# Load and parse the export.xml file
export_file_path = os.path.join(apple_health_folder, "export.xml")
tree = ET.parse(export_file_path)
root = tree.getroot()

# Display the root element and its immediate children to understand the structure
root_tag = root.tag
root_attributes = root.attrib
first_level_children = [child.tag for child in root[:10]]  # Display the first 10 tags

root_tag, root_attributes, first_level_children


In [ ]:
# Extract a sample of the first 10 'Record' elements to inspect their attributes and structure
record_elements = root.findall('Record')
sample_records = [{attr: record.get(attr) for attr in record.attrib} for record in record_elements[:10]]

# Display a sample of the records' attributes
sample_records


In [ ]:
# Filter for specific health metrics: Sleep, Steps, and Calories
# Sleep is typically a CategoryType; Steps and Calories are QuantityTypes
sleep_records = [rec.attrib for rec in record_elements if "HKCategoryTypeIdentifierSleepAnalysis" in rec.get("type", "")]
step_records = [rec.attrib for rec in record_elements if "HKQuantityTypeIdentifierStepCount" in rec.get("type", "")]
calorie_records = [rec.attrib for rec in record_elements if "HKQuantityTypeIdentifierActiveEnergyBurned" in rec.get("type", "")]

# Check counts of relevant records
len(sleep_records), len(step_records), len(calorie_records)


In [ ]:

# Function to convert date strings to datetime objects and extract the date
def extract_date(record):
    return datetime.strptime(record["startDate"].split(" ")[0], "%Y-%m-%d")

# Aggregate step data by date
steps_df = pd.DataFrame(step_records)
steps_df["startDate"] = steps_df["startDate"].apply(lambda x: x.split(" ")[0])  # Extract date only
steps_df["value"] = steps_df["value"].astype(float)  # Convert step counts to numeric
daily_steps = steps_df.groupby("startDate")["value"].sum().reset_index()
daily_steps.rename(columns={"startDate": "Date", "value": "Total_Steps"}, inplace=True)

# Aggregate calorie data by date
calories_df = pd.DataFrame(calorie_records)
calories_df["startDate"] = calories_df["startDate"].apply(lambda x: x.split(" ")[0])  # Extract date only
calories_df["value"] = calories_df["value"].astype(float)  # Convert calories to numeric
daily_calories = calories_df.groupby("startDate")["value"].sum().reset_index()
daily_calories.rename(columns={"startDate": "Date", "value": "Total_Calories_Burned"}, inplace=True)

# Display the daily aggregated data for steps and calories
daily_steps.head(), daily_calories.head()


In [ ]:
# Convert sleep data into a DataFrame for processing
sleep_df = pd.DataFrame(sleep_records)

# Extract relevant columns and convert dates
sleep_df["startDate"] = pd.to_datetime(sleep_df["startDate"])
sleep_df["endDate"] = pd.to_datetime(sleep_df["endDate"])
sleep_df["duration"] = (sleep_df["endDate"] - sleep_df["startDate"]).dt.total_seconds() / 3600  # Duration in hours

# Extract date only for aggregation
sleep_df["Date"] = sleep_df["startDate"].dt.date

# Aggregate total sleep duration per day
daily_sleep = sleep_df.groupby("Date")["duration"].sum().reset_index()
daily_sleep.rename(columns={"duration": "Total_Sleep_Hours"}, inplace=True)

# Display the aggregated daily sleep data
daily_sleep.head()


In [ ]:
# Combine daily steps, calories, and sleep into a single DataFrame
combined_data = pd.merge(daily_steps, daily_calories, on="Date", how="outer")
combined_data = pd.merge(combined_data, daily_sleep, on="Date", how="outer")

# Sort by Date and fill missing values with 0 for simplicity
combined_data["Date"] = pd.to_datetime(combined_data["Date"])
combined_data.sort_values(by="Date", inplace=True)
combined_data.fillna(0, inplace=True)

# Display the combined dataset
tools.display_dataframe_to_user(name="Aggregated Health Metrics", dataframe=combined_data)


In [ ]:
# Check for duplicate dates in the combined data
duplicates_by_date = combined_data.duplicated(subset=["Date"], keep=False)

# Filter rows with duplicate dates
duplicate_rows = combined_data[duplicates_by_date]

# Group by date and combine the rows, summing the numeric values (e.g., steps, calories, sleep hours)
merged_data = duplicate_rows.groupby("Date").sum().reset_index()

# Replace the original duplicate rows with the merged data in the dataset
non_duplicates = combined_data[~duplicates_by_date]
cleaned_combined_data = pd.concat([non_duplicates, merged_data], ignore_index=True).sort_values(by="Date")

# Display the cleaned dataset with merged rows
tools.display_dataframe_to_user(name="Cleaned and Merged Health Metrics", dataframe=cleaned_combined_data)


# **For Calculating Sleep Quality And Creating a Column For It**

In [12]:
df = pd.read_excel(f'/content/Dataset_without_sleep_quality.xlsx')
df.head()

# Define Sleep Quality
def determine_sleep_quality(row):
    if row['Total_Sleep_Hours'] >= 7 and row['REM_Sleep'] >= 1.5 and row['Deep_Sleep'] >= 1.0:
        return 'Good'
    elif row['Total_Sleep_Hours'] >= 6:
        return 'Moderate'
    else:
        return 'Poor'

df['Sleep_Quality'] = df.apply(determine_sleep_quality, axis=1)

# Save the updated dataset to a new file
df.to_excel('Updated_Sleep_Dataset_With_Quality.xlsx', index=False)


